In [3]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
from pandas import DataFrame
import statsmodels.api as sm

In [4]:
import scipy
# from datascience import *
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os
import seaborn as sns
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None

In [5]:
import numpy as np
import scipy
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#scikit-learn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import preprocessing

abstractions

In [6]:
def rmse(pred, actual):
    return np.sqrt(np.mean((pred - actual) ** 2))
def sse(pred, actual):
    return sum((pred-actual)**2)
def AIC(train, pred, actual):
    return 2 * train.shape[1] - 2 * np.log(sse(pred, actual))
def BIC(train, pred, actual):
    return train.shape[0] * np.log(sse(pred,actual)/train.shape[0]) + train.shape[1] * np.log(train.shape[0])

In [ ]:
first write a python equivalent of propublica, compare our racial result to theirs 
then write similar code on gender/marital/age... 

In [ ]:
propublica use r for model summary, we use aquitas 

#### appropriateness of model(s) for data (10)

#### explanation of model(s) and reasons for using (10)

#### data cleaning (procedures, features to extract for analysis including data reduction/transformation) (10)

#### modeling results (15)

#### explanation of results (15)

In [30]:
compas_two_year = pd.read_csv('data/compas-scores-two-years.csv')

In [31]:
display(compas_two_year_violent)

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid,two_year_recid.1
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-08-14,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,13009779CF10A,(F3),NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,NaN,1,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Recidivism,3,Low,2013-01-27,Risk of Violence,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1,1
2,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,8,High,2013-01-13,Risk of Violence,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0,0
3,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,1,0,0,2,NaN,NaN,NaN,12014130CF10A,NaN,2013-01-09,76.0,F,arrest case no charge,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-03-26,Risk of Violence,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0,0
4,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,0,1,0,0,0,0.0,2013-11-30 04:50:18,2013-12-01 12:28:56,13022355MM10A,2013-11-30,NaN,0.0,M,Battery,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-11-30,Risk of Violence,1,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0,0
5,9,steven stewart,steven,stewart,2013-08-30,Male,1973-02-25,43,25 - 45,Other,0,4,0,0,3,-1.0,2013-08-29 08:55:23,2013-08-30 08:42:13,13012216CF10A,NaN,2013-08-29,1.0,F,arrest case no charge,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,4,Low,2013-08-30,Risk of Violence,3,Low,2013-08-30,2014-05-22,2014-06-03,3,0,265,0,0,0
6,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,Caucasian,0,1,0,0,0,-1.0,2014-03-15 05:35:34,2014-03-18 04:28:46,14004524MM10A,2014-03-15,NaN,1.0,M,Battery,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2014-03-16,Risk of Violence,1,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0,0
7,13,bo bradac,bo,bradac,2013-11-04,Male,1994-06-10,21,Less than 25,Caucasian,0,3,0,0,1,428.0,2015-01-06 03:55:34,2015-01-07 03:38:44,13000017CF10A,2012-12-31,NaN,308.0,F,Insurance Fraud,1,15002891MM10A,(M1),0.0,2015-01-06,Battery,2015-01-06,2015-01-07,NaN,1,15000258CF10A,(F2),2015-01-06,Aggrav Battery w/Deadly Weapon,Risk of Recidivism,3,Low,2013-11-04,Risk of Violence,5,Medium,2013-11-04,2015-01-06,2015-01-07,1,0,428,1,1,1
8,14,benjamin franc,benjamin,franc,2013-11-26,Male,1988-06-01,27,25 - 45,Caucasian,0,4,0,0,0,-1.0,2013-11-25 06:31:06,2013-11-26 08:26:57,13016402CF10A,2013-11-25,NaN,1.0,F,"Poss 3,4 MDMA (Ecstasy)",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,4,Low,2013-11-26,Risk of Violence,4,Low,2013-11-26,2013-11-25,2013-11-26,0,0,857,0,0,0
9,16,kortney coleman,kortney,coleman,2013-01-01,Female,1978-08

https://stackoverflow.com/questions/32011359/convert-categorical-data-in-pandas-dataframe 

how to convert to category to number 

In [9]:
sample =  compas_two_year_violent.loc[:,['sex','age_cat','race','two_year_recid']]

In [10]:
display(sample)

,sex,age_cat,race,two_year_recid
0,Male,Greater than 45,Other,0
1,Male,25 - 45,African-American,1
2,Male,Less than 25,African-American,0
3,Male,25 - 45,Other,0
4,Male,25 - 45,Other,0
5,Male,25 - 45,Other,0
6,Female,25 - 45,Caucasian,0
7,Male,Less than 25,Caucasian,1
8,Male,25 - 45,Caucasian,0
9,Female,25 - 45,Caucasian,0


In [11]:
sample['sex'] = sample['sex'].astype('category')
sample['age_cat'] = sample['age_cat'].astype('category')
sample['race'] = sample['race'].astype('category')

In [12]:
cat_columns = sample.select_dtypes(['category']).columns

In [13]:
cat_columns

Index(['sex', 'age_cat', 'race'], dtype='object')

In [14]:
sample[cat_columns] = sample[cat_columns].apply(lambda x: x.cat.codes)

In [15]:
sample

,sex,age_cat,race,two_year_recid
0,1,1,5,0
1,1,0,0,1
2,1,2,0,0
3,1,0,5,0
4,1,0,5,0
5,1,0,5,0
6,0,0,2,0
7,1,2,2,1
8,1,0,2,0
9,0,0,2,0


In [16]:
X = sample.loc[:,['sex','age_cat','race']]

y = sample.loc[:,['two_year_recid']]

# set the random seed
np.random.seed(100)

# split the data: all data -> 0.8 train, 0.2 test
# train_test_split returns 4 values: X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.80, test_size=0.20)

# split the data: train -> 0.75 train, 0.25 validate
# Returns 4 values: X_train, X_validate, y_train, y_validate

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    train_size=0.75, test_size=0.25)

## cannot directly use sex, age_cat, race because they are categorical, not number 

In [20]:
# create a linear regression model
model_reg = LinearRegression()

#fit your model
model_reg = model_reg.fit(X_train, y_train)

#predict X_train using your model
pred_reg = model_reg.predict(X_train)

#predict X_val using your model
val_pred_reg = model_reg.predict(X_val)

In [68]:
OLS_r2_train

0.020345969539292685

### improve 2 dummy value

In [34]:
X = compas_two_year.loc[:,['sex','age_cat','race','priors_count.1','c_charge_degree']]
X = pd.get_dummies(X)
y = compas_two_year['decile_score.1']

# set the random seed
np.random.seed(100)

# split the data: all data -> 0.8 train, 0.2 test
# train_test_split returns 4 values: X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.80, test_size=0.20)

# split the data: train -> 0.75 train, 0.25 validate
# Returns 4 values: X_train, X_validate, y_train, y_validate

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    train_size=0.75, test_size=0.25)

In [36]:
X = sm.add_constant(X) # adding a constant

model = sm.OLS(y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:         decile_score.1   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.406
Method:                 Least Squares   F-statistic:                     493.5
Date:                Thu, 25 Apr 2019   Prob (F-statistic):               0.00
Time:                        21:38:34   Log-Likelihood:                -15925.
No. Observations:                7214   AIC:                         3.187e+04
Df Residuals:                    7203   BIC:                         3.195e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [ ]:
model = sm.Logit(y, X)
 
result = model.fit()

In [96]:
Logit_model = sm.Logit(y, X)
predictions = model.fit()

print_logit_model = model.summary()
print(print_logit_model)

ValueError: endog must be in the unit interval.

#### improve 3 predicting category instead of predicting raw score

In [37]:
display(compas_two_year)

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-08-14,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,13009779CF10A,(F3),NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,NaN,1,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Recidivism,3,Low,2013-01-27,Risk of Violence,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,13011511MM10A,(M1),0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,4,Low,2013-04-14,Risk of Violence,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,8,High,2013-01-13,Risk of Violence,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,1,0,0,2,NaN,NaN,NaN,12014130CF10A,NaN,2013-01-09,76.0,F,arrest case no charge,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-03-26,Risk of Violence,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0
5,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,0,1,0,0,0,0.0,2013-11-30 04:50:18,2013-12-01 12:28:56,13022355MM10A,2013-11-30,NaN,0.0,M,Battery,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-11-30,Risk of Violence,1,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,0,6,0,0,14,-1.0,2014-02-18 05:08:24,2014-02-24 12:18:30,14002304CF10A,2014-02-18,NaN,1.0,F,Possession Burglary Tools,1,14004485CF10A,(F2),0.0,2014-03-31,Poss of Firearm by Convic Felo,2014-03-31,2014-04-18,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,6,Medium,2014-02-19,Risk of Violence,2,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1
7,9,steven stewart,steven,stewart,2013-08-30,Male,1973-02-25,43,25 - 45,Other,0,4,0,0,3,-1.0,2013-08-29 08:55:23,2013-08-30 08:42:13,13012216CF10A,NaN,2013-08-29,1.0,F,arrest case no charge,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,4,Low,2013-08-30,Risk of Violence,3,Low,2013-08-30,2014-05-22,2014-06-03,3,0,265,0,0
8,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,Caucasian,0,1,0,0,0,-1.0,2014-03-15 05:35:34,2014-03-18 04:28:46,14004524MM10A,2014-03-15,NaN,1.0,M,Battery,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2014-03-16,Risk of Violence,1,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0
9,13,bo bradac,bo,bradac,2013-11-04,Male,1994-06-10,21,Less

In [76]:
cat = compas_two_year

In [78]:
cat.shape[0]

7214

In [82]:
cat = cat.replace({"High": 1, "Low": 0})

In [84]:
cat = cat.drop(cat[cat.score_text == "Medium"].index)

In [93]:
X_cat = cat.loc[:,['sex','age_cat','race','priors_count.1','c_charge_degree']]
X_cat = pd.get_dummies(X_cat)
y_cat = cat['score_text'].convert_objects(convert_numeric=True)

    
# set the random seed
np.random.seed(100)

# split the data: all data -> 0.8 train, 0.2 test
# train_test_split returns 4 values: X_train, X_test, y_train, y_test

X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y_cat,
                                                    train_size=0.80, test_size=0.20)

# split the data: train -> 0.75 train, 0.25 validate
# Returns 4 values: X_train, X_validate, y_train, y_validate

X_train_cat, X_val_cat, y_train_cat, y_val_cat = train_test_split(X_train_cat, y_train_cat,
                                                    train_size=0.75, test_size=0.25)

In [94]:
X_cat = sm.add_constant(X_cat) # adding a constant

model = sm.OLS(y_cat, X_cat).fit()
predictions = model.predict(X_cat) 

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:             score_text   R-squared:                       0.362
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     300.3
Date:                Thu, 25 Apr 2019   Prob (F-statistic):               0.00
Time:                        22:12:37   Log-Likelihood:                -1991.9
No. Observations:                5300   AIC:                             4006.
Df Residuals:                    5289   BIC:                             4078.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     